In [1]:
import pandas as pd
import numpy as np
from datetime import date, timedelta
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.axes as ax
import openpyxl
from openpyxl.styles import Border, Side, PatternFill, Font, GradientFill, Alignment
from openpyxl.drawing.image import Image
import win32com.client as win32
import os
from PIL import ImageGrab
import win32com.client

olApp = win32.Dispatch('Outlook.Application')
olNS = olApp.GetNameSpace('MAPI')

import sys
sys.path.append("C:\\Users\\KlimovaAnnaA\\Documents\\MyFiles\\Projects\\OCP")
from Defs import merge_SalesUnits
from Defs import merge_Mapping
from Defs import Period
from Defs import new_list
from Defs import export_from_RISKCUSTOM
from Defs import add_in_currency_column
from Defs import concat_columns

In [2]:
data_excel = pd.read_excel('dyn_invoices.xlsx')
data_excel.head(2)

c:\Users\KlimovaAnnaA\AppData\Local\miniconda3\Lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


,counterpartyName,counterpartyId,counterpartyCountryName,buCode,buName,fiDocumentNumber,documentDate,dueDate,changeDate,daysOverdue,documentCurrency,debtInDocCurrency,debtInDocCurrencyDiff,fxRateUsd,debtInUsd,debtInUsdDiff,ratingChangeDate,pdChangeDate
0,TOROS TARIM SANAYI VE TICARET A.S.,1000100935,Turkey,AECOR,Corrigo Fertilizers LLC,1800000314,2023-03-24,2023-04-23,2023-03-24,30.0,USD,1416000.0,1416000.0,1.0,1416000.0,1416000.0,30,NaN
1,TOROS TARIM SANAYI VE TICARET A.S.,1000100935,Turkey,AECOR,Corrigo Fertilizers LLC,1800000314,2023-03-24,2023-03-24,2023-03-24,0.0,USD,944000.0,944000.0,1.0,944000.0,944000.0,30,NaN


In [3]:
query = """
select distinct "rating", "letter_rating" 
from "RISKACCESS"."camExportGb"
"""
data_RISKACCESS = export_from_RISKCUSTOM(query)
data_RISKACCESS.head(2)

C:\Users\KlimovaAnnaA\Documents\MyFiles\Projects\OCP\Defs.py:93: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  data_export = pd.read_sql(query, con=connection)


,rating,letter_rating
0,60.0,None
1,25.0,None


In [65]:
data_excel_work = data_excel
data_excel_work['numeric_ratingChangeDate'] = pd.to_numeric(data_excel_work['ratingChangeDate'], errors='coerce') # get only numeric values
data_RISKACCESS_work = data_RISKACCESS.dropna()
data_excel_work_merge = data_excel_work.merge(data_RISKACCESS_work, how='left', left_on='numeric_ratingChangeDate', right_on='rating')
assert data_excel_work.shape[0] == data_excel_work_merge.shape[0] # quality of merging
assert len(data_excel_work_merge[(~data_excel_work_merge.numeric_ratingChangeDate.isna())&(data_excel_work_merge.letter_rating.isna())]) == 0 # check empty numeric values after merging
data_excel_work_merge.loc[data_excel_work_merge.numeric_ratingChangeDate.isna(),'letter_rating'] = data_excel_work_merge.loc[data_excel_work_merge.numeric_ratingChangeDate.isna(),'ratingChangeDate'] # concat columns
# fill empty data
assert data_excel_work_merge[data_excel_work_merge.letter_rating.isna()].shape[0] == data_excel[data_excel.ratingChangeDate.isna()].shape[0]
# data_excel_work_merge['Days_life'] = (data_excel_work_merge.dueDate - data_excel_work_merge.documentDate).dt.days + 1
# data_excel_work_merge['debtInUsdDiff_Days_life'] = data_excel_work_merge.debtInUsdDiff / data_excel_work_merge.Days_life
# data_excel_work_merge['debtInUsd_Days_life'] = data_excel_work_merge.debtInUsd / data_excel_work_merge.Days_life

# country table
data_excel_work_merge['index'] = data_excel_work_merge.index
data_excel_work_merge['pdChangeDate_have_data'] = ~data_excel_work_merge.pdChangeDate.isna()
data_excel_work_pivot_1 = data_excel_work_merge.pivot_table(index=['counterpartyCountryName','pdChangeDate_have_data'], values=['index','pdChangeDate'], aggfunc={'index':'count','pdChangeDate':'mean'}).reset_index().rename(columns={'index':'count_of_strings','pdChangeDate':'mean_pdChangeDate'}) # mean pd data

data_excel_work_merge['mean_pdChangeDate'] = data_excel_work_merge.reset_index(drop=True).merge(data_excel_work_pivot_1.dropna(subset='mean_pdChangeDate'), how='left', left_on='counterpartyCountryName',right_on='counterpartyCountryName')['mean_pdChangeDate'] # fill values
data_excel_work_merge.loc[data_excel_work_merge.pdChangeDate.isna(), 'pdChangeDate'] = data_excel_work_merge.loc[data_excel_work_merge.pdChangeDate.isna(), 'mean_pdChangeDate']

In [5]:
data_new_model = pd.read_excel('new_model_v4.xlsx')
data_new_model.head(2)

,##batch_id,##deleted,##inserted,##origin_name,reportDate,kundennummer,country,reportYear,reportYearPrev,inventoriesPrev,...,ratingFinal,total_revenue,costOfGoodSold,financialExpenses,inventories,netProfit,taxes,counterpartyPd,shortCreditId,counterpartyId
0,36784121@WHWEEK,N,2024-05-30 14:45:38,WHFLEX,2024-05-30,DE3306356,DE,NaN,NaN,NaN,...,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,DE3306356,U2011010381
1,36784121@WHWEEK,N,2024-05-30 14:45:38,WHFLEX,2024-05-30,SK35826487,SK,2022.0,2021.0,7.173264e+07,...,A,1.107686e+09,7.821319e+08,2.094410e+06,1.197343e+08,1.149430e+08,1.689498e+07,1.84,SK35826487,U2011071401


In [66]:
data_new_model_work = data_new_model[['ratingFinal','counterpartyPd','counterpartyId']].dropna(subset=['counterpartyId','counterpartyPd'])
data_print_merge = data_excel_work_merge.reset_index(drop=True).merge(data_new_model_work,how='left',left_on='counterpartyId',right_on='counterpartyId').rename(columns={'ratingFinal':'model_ratingFinal','counterpartyPd':'model_counterpartyPd'})
assert data_print_merge.shape[0] == data_excel_work_merge.shape[0]
# country table
# data_print_merge['model_counterpartyPd'] = pd.to_numeric(data_print_merge['model_counterpartyPd'], errors='coerce')
data_print_merge['model_counterpartyPd_have_data'] = False
data_print_merge.loc[~data_print_merge.model_counterpartyPd.isna(), 'model_counterpartyPd_have_data'] = True
data_print_pivot_2 = data_print_merge.pivot_table(index=['counterpartyCountryName','model_counterpartyPd_have_data'], values=['index','model_counterpartyPd'], aggfunc={'index':'count','model_counterpartyPd':'mean'}).reset_index().rename(columns={'index':'count_of_strings','model_counterpartyPd':'mean_model_counterpartyPd'})

data_print_merge['mean_model_counterpartyPd'] = data_print_merge.reset_index(drop=True).merge(data_print_pivot_2.dropna(subset='mean_model_counterpartyPd'),how='left',right_on='counterpartyCountryName',left_on='counterpartyCountryName')['mean_model_counterpartyPd']
data_print_merge.loc[data_print_merge.model_counterpartyPd.isna(), 'model_counterpartyPd'] = data_print_merge.loc[data_print_merge.model_counterpartyPd.isna(), 'mean_model_counterpartyPd']

In [69]:
data_print_merge.counterpartyCountryName.unique()
data_print_merge[(data_print_merge.counterpartyCountryName=='Azerbaijan')&(~data_print_merge.model_counterpartyPd.isna())]
# data_print_merge.loc[~data_print_merge.model_counterpartyPd.isna(), ['model_counterpartyPd','model_counterpartyPd_have_data']]
# data_print_merge.model_counterpartyPd.unique()
# data_new_model_work.counterpartyPd = data_new_model_work.counterpartyPd

# pd.to_numeric(data_excel_work['ratingChangeDate'], errors='coerce')
# data_print_merge
# data_print_pivot_2.dropna(subset='mean_model_counterpartyPd')
# data_print_merge.drop(['numeric_ratingChangeDate','rating','ratingChangeDate','index','pdChangeDate_have_data','model_counterpartyPd_have_data','mean_model_counterpartyPd','mean_pdChangeDate'],axis=1).rename(columns={'letter_rating':'ratingChangeDate'}) 

,counterpartyName,counterpartyId,counterpartyCountryName,buCode,buName,fiDocumentNumber,documentDate,dueDate,changeDate,daysOverdue,...,numeric_ratingChangeDate,rating,letter_rating,index,pdChangeDate_have_data,mean_pdChangeDate,model_ratingFinal,model_counterpartyPd,model_counterpartyPd_have_data,mean_model_counterpartyPd


In [70]:
print_to_excel = True
print_to_csv = True

In [72]:
if print_to_excel == True:
    
    Output_file = f'{str(date.today())}_invoices.xlsx'

    writer = pd.ExcelWriter(Output_file, engine='openpyxl')  
    workbook=writer.book
    data_excel_work_pivot_1.to_excel(writer, sheet_name='country_table', index=False)
    data_print_pivot_2.to_excel(writer, sheet_name='country_table', startcol=data_excel_work_pivot_1.shape[1]+1, index=False)
    writer.close()

In [73]:
if print_to_csv == True:

    data_print = data_print_merge.drop(['numeric_ratingChangeDate','rating','ratingChangeDate','index','pdChangeDate_have_data','model_counterpartyPd_have_data','mean_model_counterpartyPd','mean_pdChangeDate'],axis=1).rename(columns={'letter_rating':'ratingChangeDate'}) #drop columns

    data_print.to_csv(f'{str(date.today())}_invoices.csv')